# How to run multiple queries simultaneously and bind the result

To run multiple queries simultaneously and bind the result with C# and Dapper, you can use the <code>QueryMultiple</code> method, which allows you to execute multiple SQL statements in one query and return a <code>GridReader</code> object that can map the results to multiple objects. You can also use the <code>QueryMultipleAsync</code> method for asynchronous execution.

Here is a complete example using the AdventureWorks2022 database.


In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
class Product
{
    public int ProductID { get; set; }
    public string Name { get; set; }    
}

class ProductReview
{
    public int ProductReviewID { get; set; }
    public int ProductID { get; set; }
    public string ReviewerName { get; set; }
    public string EmailAddress { get; set; }
    public string Comments { get; set; }
}

In [3]:
#r "nuget:Dapper"
#r "nuget:Microsoft.Data.SqlClient"
using Microsoft.Data.SqlClient;
using System.Data;
using Dapper;

// Define the SQL queries to run
var sql = @"SELECT * FROM Production.Product WHERE ProductID = @Id;
            SELECT * FROM Production.ProductReview WHERE ProductID = @Id;";

// Define the parameters for the queries
var parameters = new { Id = 937 };

// Create a connection to the database
using (var connection = new SqlConnection(connectionString))
{
    // Open the connection
    await connection.OpenAsync();

    // Execute the queries and get the GridReader object
    using (var gridReader = await connection.QueryMultipleAsync(sql, parameters))
    {
        // Read the first result set as a Product object
        var product = await gridReader.ReadSingleAsync<Product>();

        // Read the second result set as a list of ProductReview objects
        var reviews = await gridReader.ReadAsync<ProductReview>();

        // Display the results
        Console.WriteLine($"Product: {product.Name}");
        foreach(var review in reviews)
        {
            Console.WriteLine($"Reviews: {review.ReviewerName}, {review.EmailAddress} \n\t {review.Comments}");
        }
    }
}

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.2

Product: HL Mountain Pedal
Reviews: David, david@graphicdesigninstitute.com 
	 A little on the heavy side, but overall the entry/exit is easy in all conditions. I've used these pedals for 
more than 3 years and I've never had a problem. Cleanup is easy. Mud and sand don't get trapped. I would like 
them even better if there was a weight reduction. Maybe in the next design. Still, I would recommend them to a friend.
Reviews: Jill, jill@margiestravel.com 
	 Maybe it's just because I'm new to mountain biking, but I had a terrible time getting use
to these pedals. In my first outing, I wiped out trying to release my foot. Any suggestions on
ways I can adjust the pedals, or is it just a learning curve thing?
